In [1]:
import sys, os

# Add utility_scripts in the current path so that they can be imported directly just like in interactive mode
sys.path.append(os.path.abspath("../usr/lib/"))
for script_folder in os.listdir("../usr/lib/"):
    sys.path.append(os.path.abspath("../usr/lib/"+script_folder))

In [2]:
from datetime import date
from statistics import mean, median
import pickle

import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
from torch import nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

from lastquerytransformer import Riiid
from riiidutils import RiiidDataset, riiid_collate_fn, riiid_collate_fn_right_padding

In [3]:
loc = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','Localhost')
if loc == 'Interactive' or loc == 'Localhost':
    conf = {
        'train_size': 96_000,
        'batch_size': 128,
        "max_length": 1728,
        # model
        "dropout": 0.1,
        # training loop
        'epochs': 50,
        'eval_steps': 250,
        # adam
        'learning_rate': 1e-3,
        'epsilon': 1e-8,
        "weight_decay": 0.01,
        # clip_grad_value
        "clip_value": False
    }
# When it is run after an api push.
elif loc == 'Batch':
    conf = {
        'train_size': 400_000,
        'batch_size': 256,
        "max_length": 1728,
        # model
        "dropout": 0.1,
        # training loop
        'epochs': 40,
        'eval_steps': 500,
        # adam
        'learning_rate': 1e-3,
        'epsilon': 1e-8,
        "weight_decay": 0.01,
        # clip_grad_value
        "clip_value": False
    }

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Introduction

Le fichier train.csv comprend un peu plus de 100 millions de lignes.  
Il y a un peu moins de 400 mille user_id uniques.  
Le modèle utilisé dans ce notebook prend en entrée une série par utilisateur.

# Features utilisées
Pour chaque question de la série d'apprentissage d'un utilisateur quelconque  

1. Question ID: correspond à content_id (lorsque l'élément est une question).
2. Question part: correspond à part dans question.csv pour l'élément dont question_id correspondant à content_id
3. Answer correctness: valeur de answered_correctly de l'exemple (ou target encoding de la question ?)
4. Current question elapsed time: prior question de la question suivante.
5. Timestamp difference: current question timestamp - timestamp of the last question from the same user

# Chargement des données et instanciation des datasets

In [5]:
with open('../input/riiid-sequences/users_y.pickle', 'rb') as f:
    users_y = pickle.load(f)
with open('../input/riiid-sequences/users_cat.pickle', 'rb') as f:
    users_cat = pickle.load(f)
with open('../input/riiid-sequences/users_cont.pickle', 'rb') as f:
    users_cont = pickle.load(f)

In [6]:
seed = 12
cat_train, cat_val, cont_train, cont_val, y_train, y_val = train_test_split(users_cat, users_cont, users_y, test_size=.05, random_state=seed)

In [7]:
cat_train = cat_train[:conf['train_size']]
cont_train = cont_train[:conf['train_size']]
y_train = y_train[:conf['train_size']]

In [8]:
print("Number of train examples:", len(y_train))
print("Number of valid examples:", len(y_val))
print("Train set answered_correctly average value:", "{:.3}".format(y_train.mean()))
print("Valid set answered_correctly average value:", "{:.3}".format(y_val.mean()))
print("Train set median sequence length:", "{:.0f}".format(median([user_seq.shape[0] for user_seq in cat_train])))
print("Valid set median sequence length:", "{:.0f}".format(median([user_seq.shape[0] for user_seq in cat_val])))

Number of train examples: 96000
Number of valid examples: 19683
Train set answered_correctly average value: 0.472
Valid set answered_correctly average value: 0.475
Train set median sequence length: 40
Valid set median sequence length: 41


## Batches
Dynamic Padding: ajout de padding batch par batch pour avoir une même longueur de séquence dans chaque batch.  
Uniform size batching: on trie les utilisateurs par longueur de séquence, afin d'avoir des longueurs plus proches dans chaque batch  

Afin de mettre en œuvre ces deux stratégies on va par simplicité trier au préalable et conjointement les listes batch_cat, batch_cont et batch_y par longueur des séquences dans batch_cat/batch_cont (c'est RiiidDataset qui s'en charge). Le DataLoader utilisera une fonction collate_fn permettant d'ajouter du padding dynamiquement batch par batch.

In [9]:
# train_dataset = RiiidDataset(cat_train, cont_train, y_train, sort_sequences=False)
# val_dataset = RiiidDataset(cat_val, cont_val, y_val, sort_sequences=False)
train_dataset = RiiidDataset(cat_train, cont_train, y_train, sort_sequences=True, max_length=conf["max_length"])
val_dataset = RiiidDataset(cat_val, cont_val, y_val, sort_sequences=True, max_length=conf["max_length"])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=conf['batch_size'], shuffle=False, collate_fn=riiid_collate_fn, drop_last=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4*conf['batch_size'], shuffle=False, collate_fn=riiid_collate_fn, pin_memory=True)
# train_loader = DataLoader(train_dataset, batch_size=conf['batch_size'], shuffle=False, collate_fn=riiid_collate_fn_right_padding, drop_last=True, pin_memory=True)
# val_loader = DataLoader(val_dataset, batch_size=4*conf['batch_size'], shuffle=False, collate_fn=riiid_collate_fn_right_padding, pin_memory=True)

Without dynamic batching

In [11]:
# cat_train = pad_sequence_left([torch.tensor(el) for el in cat_train], batch_first=True)
# cont_train = pad_sequence_left([torch.tensor(el, dtype=torch.float) for el in cont_train], batch_first=True)
# #y = torch.tensor(y, dtype=torch.float)
# cat_val = pad_sequence_left([torch.tensor(el) for el in cat_val], batch_first=True)
# cont_val = pad_sequence_left([torch.tensor(el, dtype=torch.float) for el in cont_val], batch_first=True)
# #y = torch.tensor(y, dtype=torch.float)

# train_dataset = RiiidDataset(cat_train, cont_train, y_train, sort_sequences=False)
# val_dataset = RiiidDataset(cat_val, cont_val, y_val, sort_sequences=False)

# train_loader = DataLoader(train_dataset, batch_size=conf['batch_size'], shuffle=False, drop_last=True)
# val_loader = DataLoader(val_dataset, batch_size=4*conf['batch_size'], shuffle=False)

# Modèle
Pour le modèle, l'auteur s'est inspiré de la solution arrivée 3è à la compétition data Science bowl 2019 ([discussion](https://www.kaggle.com/c/data-science-bowl-2019/discussion/127891), [code](https://www.kaggle.com/limerobot/dsb2019-v77-tr-dt-aug0-5-3tta))  
La procédure de création des embeddings est expliquée, on suppose que l'auteur s'est basé dessus.
## Embeddings
On utilise un embedding catégoriel pour les 3 premières variables et un embedding continu pour les 2 dernières.

In [12]:
maximums = {'question_id': 13523, 'part': 7, 'answered_correctly': 3}
model = Riiid(maximums, dropout=conf["dropout"]).to(device)
optimizer = optim.AdamW(model.parameters(), lr=conf['learning_rate'], eps=conf['epsilon'], weight_decay=conf["weight_decay"])
# optimizer = optim.SGD(model.parameters(), lr=conf["learning_rate"], momentum=0.9)
# scheduler = ReduceLROnPlateau(optimizer, "min")
criterion = nn.BCEWithLogitsLoss()
scaler = GradScaler()

completed_epochs = 0
step = 0

## Loading

In [13]:
# checkpoint = torch.load('../input/lastquerytransformer40ebundlefix/lqt-2021-07-25.pt')
# completed_epochs = checkpoint['epoch']
# step = checkpoint["step"]
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [14]:
# checkpoint["model_state_dict"]["emb.answer_emb.weight"]

# Training

In [15]:
@torch.no_grad()
def evaluate(model, criterion, dataloader):
    model.eval()
    losses = []
    accuracies = []
    y_true = []
    y_score = []
    for i, batch in enumerate(dataloader):
        x_cat = batch['cat'].to(device)
        x_cont = batch['cont'].to(device)
        y = batch['y'].to(device)
        if "lengths" in batch:
            seq_lengths = batch['lengths']
        else:
            seq_lengths = None
        ypred = model(x_cat, x_cont, seq_lengths).squeeze(1)
        loss = criterion(ypred, y)

        losses.append(float(loss))
        accuracies.append((torch.round(torch.sigmoid(ypred)) == y).float().mean().item())
        y_true.extend(y.cpu().numpy())
        y_score.extend(torch.sigmoid(ypred).cpu().numpy())

    return mean(losses), mean(accuracies), roc_auc_score(y_true, y_score)

In [16]:
print(evaluate(model, criterion, train_loader))
print(evaluate(model, criterion, val_loader))
# Time to run on laptop with 100k examples (train)
# max_len=   0, sort_seq=True, shuffle=False, bs=16: 58 s
# max_len=   0, sort_seq=False, shuffle=True, bs=16: 408 s
# max_len=1728, sort_seq=True, shuffle=False, bs=16: 99 s
# max_len=1728, sort_seq=True, shuffle=False, bs=32: 36 s
# max_len=1728, sort_seq=True, shuffle=False, bs=64: 32 s
# max_len=1728, sort_seq=True, shuffle=False, bs=128: 27 s
# max_len=1728, sort_seq=False, shuffle=True, bs=128: 200 s

(0.7211155409018198, 0.4557395833333333, 0.4390699219409951)
(0.722421088279822, 0.4519934531969902, 0.4317338440650058)


In [17]:
log_dir = "tensorboard"

In [18]:
if log_dir is not None:
    writer = SummaryWriter(log_dir)

for e in range(completed_epochs, completed_epochs+conf['epochs']):
    print("Epoch ", e)
    for batch in train_loader:
        #print(scaler.get_scale())
        model.train()
        x_cat = batch['cat'].to(device)
        x_cont = batch['cont'].to(device)
        y = batch['y'].to(device)
        if "lengths" in batch:
            seq_lengths = batch['lengths']
        else:
            seq_lengths = None
        with autocast(enabled=False):
            ypred = model(x_cat, x_cont, seq_lengths).squeeze(1)
            if ypred.isnan().sum() or ypred.isinf().sum():
                print("Nan value in output!")
            loss = criterion(ypred, y)
            if loss.isnan().sum() or loss.isinf().sum():
                print("Nan value in loss!")
        model.zero_grad()
        loss.backward()
        #scaler.scale(loss).backward()
        
        # unscale to apply gradient clipping
        #scaler.unscale_(optimizer)
        if conf["clip_value"]:
            nn.utils.clip_grad_value_(model.parameters(), clip_value=conf["clip_value"])
        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        #scaler.step(optimizer)
        #scaler.update()

        step += 1
        if step % conf['eval_steps'] == 0:
            train_loss, train_acc, train_auc = evaluate(model, criterion, train_loader)
            print("Step", step, end="\n")
            print("Train loss:", "{:.3f}".format(train_loss), end=" ")
            print("Train accuracy:", "{:.3f}".format(train_acc), end=' ')
            print("Train AUC:", "{:.3f}".format(train_auc), end='\n')
            if log_dir is not None:
                writer.add_scalar("train/loss", train_loss, step)
                writer.add_scalar("train/accuracy", train_acc, step)
                writer.add_scalar("train/auc", train_auc, step)
            if val_loader is not None:
                val_loss, val_acc, val_auc = evaluate(model, criterion, val_loader)
                print("Valid loss:", "{:.3f}".format(val_loss), end=" ")
                print("Valid accuracy:", "{:.3f}".format(val_acc), end=" ")
                print("Valid AUC:", "{:.3f}".format(val_auc), end='\n')
                if log_dir is not None:
                    writer.add_scalar("eval/loss", val_loss, step)
                    writer.add_scalar("eval/acc", val_acc, step)
                    writer.add_scalar("eval/auc", val_auc, step)
                #scheduler.step(val_loss)

if log_dir is not None:
    writer.close()


Epoch  0
Step 250
Train loss: 0.641 Train accuracy: 0.627 Train AUC: 0.680
Valid loss: 0.645 Valid accuracy: 0.622 Valid AUC: 0.674
Step 500
Train loss: 0.643 Train accuracy: 0.625 Train AUC: 0.679
Valid loss: 0.646 Valid accuracy: 0.622 Valid AUC: 0.675
Step 750
Train loss: 0.668 Train accuracy: 0.580 Train AUC: 0.669
Valid loss: 0.670 Valid accuracy: 0.574 Valid AUC: 0.659
Epoch  1
Step 1000
Train loss: 0.640 Train accuracy: 0.629 Train AUC: 0.686
Valid loss: 0.646 Valid accuracy: 0.623 Valid AUC: 0.679
Step 1250
Train loss: 0.636 Train accuracy: 0.635 Train AUC: 0.688
Valid loss: 0.642 Valid accuracy: 0.624 Valid AUC: 0.679
Step 1500
Train loss: 0.651 Train accuracy: 0.602 Train AUC: 0.682
Valid loss: 0.658 Valid accuracy: 0.592 Valid AUC: 0.666
Epoch  2
Step 1750
Train loss: 0.628 Train accuracy: 0.647 Train AUC: 0.703
Valid loss: 0.638 Valid accuracy: 0.630 Valid AUC: 0.686
Step 2000
Train loss: 0.627 Train accuracy: 0.646 Train AUC: 0.702
Valid loss: 0.639 Valid accuracy: 0.623 V

In [17]:
torch.save({
            'epoch': e,
            "step": step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss,
            }, "lqt-"+str(date.today())+".pt")

# Error analysis

In [16]:
# train_dataset[0]

(array([[4887,    5,    3],
        [4033,    5,    1],
        [5928,    5,    1],
        ...,
        [9820,    5,    2],
        [9507,    5,    1],
        [8874,    5,    2]]),
 array([[249000.,      0.],
        [ 18000.,  21186.],
        [ 31000.,  33844.],
        ...,
        [ 44000.,  78052.],
        [ 17000.,  68609.],
        [     0., 213212.]]),
 1,
 17609)

In [29]:
# batch = next(iter(train_loader)) 

In [36]:
# x_cat = batch['cat'].to(device)
# x_cont = batch['cont'].to(device)
# y = batch['y'].to(device)
# seq_lengths = batch['lengths']#.to(device)

In [153]:
# x_cat_copy = x_cat.clone()
# x_cont_copy = x_cont.clone()

In [167]:
# x_cont_copy[0, -5:, 1]

tensor([2.2320e+05, 5.0000e+01, 5.0000e+01, 5.0000e+01, 5.0000e+01],
       device='cuda:0')

In [169]:
# x_cont_copy[0, -5, 1] = 50

In [170]:
# model(x_cat_copy, x_cont_copy, seq_lengths)

tensor([[1.8957]], device='cuda:0', grad_fn=<AddmmBackward>)